***I affirm that I will not plagiarize, use unauthorized materials, or give or receive illegitimate help on assignments, papers, or examinations. I will also uphold equity and honesty in the evaluation of my work and the work of others. I do so to sustain a community built around this Code of Honor.***

If you agree to abide by this honor code please enter your name in the box below

Mengxue Tang

**Q1)** The  webpage "https://www.atptour.com/en/news/story-of-the-season" lists the top stories of the ATP tennis season this year in Singles \& Doubles. 
Write a program that uses Beautifulsoup and webscraping to populate a dataframe with columns ***Headline, Winner, Loser, Score*** that parses these stories as follows: <tab><tab>**[40 Points]**
    
- Find all the story links that appear in the landing page<br>
- For each link visit the corresponding page and grab the title and look for the presence of a '/' in the title to determine if this story pertains to a doubles or singles match. This is not foolproof but will suffice.<br>
- The names of the players are generally in the first or second paragraph of the story so don't bother parsing beyond the second para. Also assume that the winner(s) will be mentioned before the loser(s). <br>
- The score should also appear in the first or second paragraph. 
- For doubles matches, the *Winner* and *Loser* columns of your dataframe should contain the names of the two players on the corresponding team, separated  a '/' 

***Notes***

Use the *cloudscraper* library to get around captcha issues that will otherwise prevent you from programmatically accessing the data on the page. Here's some code to get you started:<br>
!pip install cloudscraper<br>
url = "https://www.atptour.com/en/news/story-of-the-season" <br>
scraper = cloudscraper.create_scraper()<br>
response = scraper.get("https://www.atptour.com/en/news/story-of-the-season") <br>


Your program should return a dataframe with the four columns duly populated with the ***headline, winner(s), loser(s), score*** of each story that appears. 



In [115]:
import requests
from bs4 import BeautifulSoup
#!pip install cloudscraper
import cloudscraper

---------------------- NEW ATTEMPT-----------------------------------------------------------------------------

In [93]:
def get_pages(url):
    import requests
    from bs4 import BeautifulSoup
    import cloudscraper
    scraper = cloudscraper.create_scraper()
    response = scraper.get(url) 
    page_soup = BeautifulSoup(response.content,'lxml')
    ### we often firstly find the core keyword, like ['h3','listing-title'] here, and form tags
    h3_tags=page_soup.find_all('h3',class_='listing-title')
    all_links=[]
    all_titles=[]
    root='https://www.atptour.com'
    for tag in h3_tags:
        ### if there're some other unrelated lines, we could find and then continue to ignore
        
        ### if we could find the actual line we want, here we still use try...except...
        try:
            link=tag.find('a').get('href')
            title=tag.find('a').get_text().strip()
            all_links.append(root+link)
            all_titles.append(title)
        except:
            pass
    return all_links, all_titles

In [94]:
url="https://www.atptour.com/en/news/story-of-the-season"
links,titles=get_pages(url)
links

['https://www.atptour.com/en/news/tsitsipas-rublev-monte-carlo-2021-final',
 'https://www.atptour.com/en/news/mektic-pavic-evans-skupski-monte-carlo-2021-doubles-final',
 'https://www.atptour.com/en/news/carreno-busta-munar-marbella-2021-final',
 'https://www.atptour.com/en/news/sonego-djere-cagliari-2021-final',
 'https://www.atptour.com/en/news/behar-escobar-marbella-2021-doubles-final',
 'https://www.atptour.com/en/news/sonego-vavassori-cagliari-2021-doubles-final',
 'https://www.atptour.com/en/news/hurkacz-sinner-miami-2021-day-12-sunday-final',
 'https://www.atptour.com/en/news/miami-2021-doubles-final',
 'https://www.atptour.com/en/news/tsitsipas-zverev-acapulco-2021-saturday',
 'https://www.atptour.com/en/news/acapulco-2021-doubles-final-granollers-zeballos-skupski-skupski',
 'https://www.atptour.com/en/news/karatsev-harris-dubai-2021-final',
 'https://www.atptour.com/en/news/cabal-farah-dubai-2021-doubles-final',
 'https://www.atptour.com/en/news/bolelli-gonzalez-santiago-2021-

In [95]:
titles

['Tsitsipas Wins Maiden Masters 1000 Crown In Monte-Carlo',
 'Mektic/Pavic Clinch Monte-Carlo Crown; Fifth Doubles Title Of Year',
 'Carreno Busta Wins First Title At Home In Marbella',
 'Sonego Fights Back To Complete Title Double In Cagliari',
 'Behar/Escobar Capture Marbella Doubles Crown',
 'Sonego Clinches Cagliari Title With Vavassori; Now For Singles Crown',
 'Pole Position: Hurkacz Claims First Masters 1000 Title In Miami',
 'Mektic/Pavic Make History, Storm To Miami Title',
 'Zverev Clinches Acapulco Title In Tsitsipas Classic',
 'Skupski Brothers Best Granollers/Zeballos For Acapulco Trophy',
 'Karatsev Completes Stunning Run, Lifts Maiden Title In Dubai',
 'Cabal/Farah Capture Dubai Trophy',
 'Bolelli/Gonzalez Seal Santiago Doubles Title',
 'Santiago Native Garin Bests Bagnis To Win Home Title',
 'Medvedev Ends Herbert’s Run, Earns 10th Title',
 'Medvedev Ends Herbert’s Run, Earns 10th Title',
 'Glasspool/Heliovaara Claim Maiden ATP Tour Crown',
 'Rublev/Karatsev Win Doha Do

In [100]:
def get_player_and_score(link,title):
    import requests
    from bs4 import BeautifulSoup
    import cloudscraper
    import re
    
    winner=[]
    loser=[]
    score=[]
    
    for i in range(len(link)):
        link0=link[i]
        title0=title[i]
        scraper = cloudscraper.create_scraper()
        response = scraper.get(link0) 
        page_soup = BeautifulSoup(response.content,'lxml')
        
        player=[]
        score0=[]
        
        #### double player
        if title0.find('/')>=0:
            p=page_soup.find_all('p')
            
            ### tranverse the first 2 passages
            for j in range(0,3):
                ### dig out the score
                s=p[j].get_text()
                score0+=re.findall('\d\-\d',s)
                
                ### get the player
                link1=p[j].find_all('a')
                for item in link1:
                    if item.get('href').find('players')>=0:
                        player.append(item.get_text())
                    else:
                        continue
                        
            winner.append(str(player[0])+'/'+str(player[1]))
            print('winner',winner)
            loser.append(str(player[2])+'/'+str(player[3]))
            score.append(score0)
        
        
        #### single player
        else:
            p=page_soup.find_all('p')
            ### tranverse first two passages
            for k in range(0,3):
                ### dig out the score
                s=p[k].get_text()
                score0+=re.findall('\d\-\d',s)
                ### get the player
                link1=p[k].find_all('a')
                for item in link1:
                    if item.get('href').find('players')>=0:
                        player.append(item.get_text())
                    else:
                        continue
                     
                        
            print('player',player)
            winner.append(player[0])
            
            loser.append(player[1])
            score.append(score0)
    
    
    return winner, loser, score
        

In [101]:
winner,loser,score=get_player_and_score(links,titles)
#### the 9th article doesn't have player in link..... strange

player ['Stefanos Tsitsipas', 'Andrey Rublev']
winner ['Stefanos Tsitsipas', 'Nikola Mektic/Mate Pavic']
player ['Pablo Carreno Busta', 'Jaume Munar']
player ['Lorenzo Sonego', 'Laslo Djere']
winner ['Stefanos Tsitsipas', 'Nikola Mektic/Mate Pavic', 'Pablo Carreno Busta', 'Lorenzo Sonego', 'Ariel Behar/Gonzalo Escobar']
player ['Lorenzo Sonego', 'Andrea Vavassori', 'Simone Bolelli', 'Andres Molteni', 'Daniele Bracciali', 'Giorgio Galimberti']
player ['Hubert Hurkacz', 'Jannik Sinner']
winner ['Stefanos Tsitsipas', 'Nikola Mektic/Mate Pavic', 'Pablo Carreno Busta', 'Lorenzo Sonego', 'Ariel Behar/Gonzalo Escobar', 'Lorenzo Sonego', 'Hubert Hurkacz', 'Nikola Mektic/Mate Pavic']
player []


IndexError: list index out of range

In [98]:
link0='https://www.atptour.com/en/news/sonego-djere-cagliari-2021-final'
scraper = cloudscraper.create_scraper()
response = scraper.get(link0) 
page_soup = BeautifulSoup(response.content,'lxml')
p=page_soup.find_all('p')
player=[]
for i in range(1,3):
    link1=p[i].find_all('a')
    for item in link1:
        if item.get('href').find('players')>=0:
            player.append(item.get_text())
        else:
            continue
player

['Laslo Djere']

In [99]:
p

[<p><a href="https://www.atptour.com/en/players/lorenzo-sonego/su87/overview">Lorenzo Sonego</a> became the first Italian in 15 years to capture an ATP Tour title on home soil on Sunday at the <a href="https://www.atptour.com/en/tournaments/cagliari/9481/overview">Sardegna Open</a>.</p>,
 <p>The third seed dug deep in the latter stages of the second set en route to a 2-6, 7-6(5), 6-4 victory over <a href="https://www.atptour.com/en/players/laslo-djere/db63/overview">Laslo Djere</a> of Serbia in just over three hours at the Tennis Club Cagliari.</p>,
 <p>“The match was long and I remained calm in the second-set tie-break,” Sonego told ATPTour.com. “I knew the match wasn’t finished because I had always kept fighting. It’s wonderful to win the title with all my team here. Now, I am really tired, but I am very happy. I am happy to follow in [Filippo] Volandri’s footsteps and win a title on home soil.”</p>,
 <p>Sonego, who ended Djere’s nine-match winning streak in Italy, is also the first 

In [59]:
link1=p[1].find_all('a')
#link_play=p[1].find_all('a')
t=p[3].get_text()
import re
re.findall('\d\-\d',t)


[]

----------------------------------------------------------------------------------------------------------------

In [120]:
url = "https://www.atptour.com/en/news/story-of-the-season" 
scraper = cloudscraper.create_scraper()
response = scraper.get("https://www.atptour.com/en/news/story-of-the-season") 
results_page = BeautifulSoup(response.content,'lxml')
results_page

<!DOCTYPE html>
<!-- START : /modules/global/head --><!--[if lt IE 7]>
    <html class="no-js lt-ie10 lt-ie9 lt-ie8 lt-ie7 ">
<![endif]--><!--[if IE 7]>
    <html class="no-js lt-ie10 lt-ie9 lt-ie8 ">
<![endif]--><!--[if IE 8]>
    <html class="no-js lt-ie10 lt-ie9 ">
<![endif]--><!--[if IE 9]>
    <html class="no-js lt-ie10 ">
<![endif]--><!--[if gt IE 9]><!--><html class="no-js" translate="no">
<!--<![endif]-->
<head>
<!-- disable auto format for telephone numbers -->
<meta content="telephone=no" name="format-detection"/>
<title>
        Story of the Season | ATP Tour | Tennis
    </title>
<meta content="initial-scale=1.0, width=768, user-scalable=yes, minimum-scale=1.0, maximum-scale=1.25" name="viewport"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"2d2d3e5431","applicationID":"148914409",

In [131]:
soup=results_page.find('div',class_='main-content')
soup1=soup.find_all('h3',class_='listing-title')
links=[]
root='https://www.atptour.com'
for item in soup1:
    links.append(root+item.find('a').get('href'))
links


['https://www.atptour.com/en/news/djokovic-medvedev-australian-open-2021-final',
 'https://www.atptour.com/en/news/dodig-polasek-australian-open-2021-doubles-final',
 'https://www.atptour.com/en/news/mektic-pavic-murray-river-open-2021-doubles-final',
 'https://www.atptour.com/en/news/great-ocean-road-open-2021-doubles-final',
 'https://www.atptour.com/en/news/auger-aliassime-evans-murray-river-open-2021-final',
 'https://www.atptour.com/en/news/sinner-travaglia-great-ocean-road-open-2021-sunday',
 'https://www.atptour.com/en/news/medvedev-berrettini-russia-italy-atp-cup-2021-final',
 'https://www.atptour.com/en/news/korda-hurkacz-delray-beach-2021-final-wednesday',
 'https://www.atptour.com/en/news/behar-escobar-delray-beach-2021-doubles-final-wednesday',
 'https://www.atptour.com/en/news/de-minaur-bublik-antalya-2021-final-wednesday',
 'https://www.atptour.com/en/news/mektic-pavic-antalya-2021-doubles-final-tuesday']

In [185]:
headline=[]
winner=[]
loser=[]
score=[]
count=0
for l in links:
    url = l 
    scraper = cloudscraper.create_scraper()
    response = scraper.get(url) 
    results_page = BeautifulSoup(response.content,'lxml')
    soup=results_page.find('div',class_='article-content')
    soup1=soup.find('h1',class_='article-title').get_text()
    headline.append(soup1)
    soup2=soup.find('div',class_='article-section').find('div',class_='article-copy')
    pars=soup2.find_all('p')[0:2]
    if soup1.find('/')!=-1:
        a=pars[0].find_all('a')[0].get_text()
        b=pars[0].find_all('a')[1].get_text()
        c=pars[1].find_all('a')[0].get_text()
        d=pars[1].find_all('a')[1].get_text()
        count+=1
        print(count)
        winner.append('{}'.format(a)+'/'+'{}'.format(b))
        loser.append('{}'.format(c)+'/'+'{}'.format(d))
    else:
        winner.append('{}'.format(pars[0].find('a').get_text()))
        loser.append('{}'.format(pars[1].find('a').get_text()))
        for p in pars:
            if p.get_text().find('-')!=-1:
                index=p.get_text().find('-')
                score.append(p.get_text()[index-1:index+2])

            else:
                continue
    
    

1


IndexError: list index out of range

In [186]:

url = 'https://www.atptour.com/en/news/mektic-pavic-murray-river-open-2021-doubles-final'
scraper = cloudscraper.create_scraper()
response = scraper.get(url) 
results_page = BeautifulSoup(response.content,'lxml')
soup=results_page.find('div',class_='article-content')
soup1=soup.find('h1',class_='article-title').get_text()
soup1.split(' ')[0]
soup2=soup.find('div',class_='article-section').find('div',class_='article-copy')
pars=soup2.find_all('p')[0:2]
a=pars[0].find_all('a')[0].get_text()
b=pars[0].find_all('a')[1].get_text()
c=pars[1].find_all('a')[0].get_text()
d=pars[1].find_all('a')[1].get_text()
a
b
c
d

IndexError: list index out of range

**Q2)** Widgets Inc. wants to run a marketing campaign targeting customers who have **not** placed any orders with the company in 2020. As the Chief Data Officer you are tasked with coming up with this list of customers. You ask your assistant to send you two data files: **[20 Points]**<br>

*Customers.csv* which lists all customers of Widgets Inc.<br>
*Orders.csv* which lists all orders placed in 2019 & 2020. <br>

**Company_ID** in  *Customers.csv*  is the same attribute as **Customer_ID** in  *Orders.csv*

How will you use the data in these two files to compile the customer list?   

***Note: Your list should only contain the names of customers***



In [2]:
import pandas as pd
import numpy as np
customer=pd.read_csv('Customers.csv',index_col='Company_ID')
order=pd.read_csv('Orders.csv',index_col='Customer_ID')
order_name=list(order.index.unique())
no_order=[]
for item in customer.index:
    if item in order_name:
        continue
    else:
        no_order.append(customer.loc[item].values[0])
no_order


['ABIOMED Inc',
 'Accenture plc',
 'Activision Blizzard',
 'Advanced Micro Devices Inc',
 'Advance Auto Parts',
 'AFLAC Inc',
 'Agilent Technologies Inc',
 'Air Products & Chemicals Inc',
 'Akamai Technologies Inc',
 'Alaska Air Group Inc',
 'Alexandria Real Estate Equities',
 'Alexion Pharmaceuticals',
 'Align Technology',
 'Allegion',
 'Alliant Energy Corp',
 'Allstate Corp',
 'Alphabet Inc.\xa0(Class A)',
 'Alphabet Inc.\xa0(Class C)',
 'Altria Group Inc',
 'Amazon.com Inc.',
 'Ameren Corp',
 'American Airlines Group',
 'American Electric Power',
 'American Express Co',
 'American International Group',
 'American Tower Corp.',
 'American Water Works Company Inc',
 'AmerisourceBergen Corp',
 'AMETEK Inc.',
 'Amgen Inc.',
 'Analog Devices, Inc.',
 'ANSYS',
 'Anthem',
 'Aon plc',
 'A.O. Smith Corp',
 'Applied Materials Inc.',
 'Archer-Daniels-Midland Co',
 'Arista Networks',
 'Arthur J. Gallagher & Co.',
 'Assurant',
 'Atmos Energy',
 'Automatic Data Processing',
 'AutoZone Inc',
 'Ava

**Q3. [10 Points]**

Given an array **ax**:<br>
[[ 1  2  3]<br>
 [ 4  5  6]<br>
 [ 7  8  9]<br>
 [10 11 12]]<br>
 
and an array **ay**:

[0 1 2 3 4]

How would you use array reshaping and broadcasting to combine **ax** and **ay** to produce the array **az** below?  

        [[[ 0,  1,  2,  3,  4],
        [ 0,  4,  8, 12, 16],
        [ 0,  7, 14, 21, 28],
        [ 0, 10, 20, 30, 40]],

       [[ 0,  2,  4,  6,  8],
        [ 0,  5, 10, 15, 20],
        [ 0,  8, 16, 24, 32],
        [ 0, 11, 22, 33, 44]],

       [[ 0,  3,  6,  9, 12],
        [ 0,  6, 12, 18, 24],
        [ 0,  9, 18, 27, 36],
        [ 0, 12, 24, 36, 48]]]
       
       
 

In [4]:
x=[[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
ax=np.array(x)
ax1=np.transpose(ax)
ax1=ax1.repeat([5,5,5],axis=0)
y=[0,1,2,3,4]
ay=np.array(y)
ay1=np.array([ay,ay,ay,ay])

ax2=np.array([np.transpose(ax1[0:5]),np.transpose(ax1[5:10]),np.transpose(ax1[10:15])])
ay2=np.array([ay1,ay1,ay1])
res=ax2*ay
res


array([[[ 0,  1,  2,  3,  4],
        [ 0,  4,  8, 12, 16],
        [ 0,  7, 14, 21, 28],
        [ 0, 10, 20, 30, 40]],

       [[ 0,  2,  4,  6,  8],
        [ 0,  5, 10, 15, 20],
        [ 0,  8, 16, 24, 32],
        [ 0, 11, 22, 33, 44]],

       [[ 0,  3,  6,  9, 12],
        [ 0,  6, 12, 18, 24],
        [ 0,  9, 18, 27, 36],
        [ 0, 12, 24, 36, 48]]])